In [11]:
!pip install langchain_groq #groq
!pip install langchain_openai #open IA
!pip install langchain_core #core

In [14]:
import os

os.environ["GROQ_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""


from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

# Initialize Groq LLM
llm = ChatGroq(
    model_name="deepseek-r1-distill-llama-70b",
    temperature=0.7
)

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]


In [13]:
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Defina la información que quieras extraer


*   Usaremos Pydanic para definir el esquema para extraer informacion personal
*   Pydantic es una biblioteca de Python que se utiliza para la validación de datos. Ayuda a garantizar que los datos que recibe tu programa coincidan con el formato esperado y proporciona mensajes de error útiles cuando los datos no se ajustan a tus especificaciones. En esencia, Pydantic te permite asegurar que las estructuras de datos en Python cumplan con tipos y restricciones específicos, lo que hace que tu código sea más robusto y resistente a errores.
*   Documenta los atributos y el propio esquema: Esta información se envía al LLM y se utiliza para mejorar la calidad de la extracción de información.
* Al usar **Optional** en las sugerencias de tipo, indicas que una variable puede ser de un tipo específico o None.





# Definamos los datos que queremos extraer de una persona.
A continuación, se recomienda escribir una cadena de documentación explicativa (comentarios) para que el modelo de chat comprenda qué datos queremos extraer.

In [18]:
from typing import Optional #Permite que un campo sea opcional

from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # Este doc-string es enviado al LLM como una descripcion del esquema persona
    # y puede ayudar a mejorar los resultados de la extracción

    # Tener en cuenta que:
    # 1. cada campo es 'optional' -- esto permite que el modelo no lo extraiga
    # 2. cada campo tiene una `description` que es utilizada por el LLM.
    # una buena descripcion puede ayudar a mejorar los resultados de la extracción
    name: Optional[str] = Field(
        default=None, description="The name of the person"
    )
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

#Definimos el "Extractor"
Nuestro extractor será una cadena con la plantilla de indicaciones y un modelo de chat con las instrucciones de extracción.

In [16]:
# Define una solicitud personalizada para proporcionar instrucciones y contexto adicional.
# 1) Puedes añadir ejemplos a la plantilla de solicitud para mejorar la calidad de la extracción.
# 2) Puedes introducir parámetros adicionales para tener en cuenta el contexto (por ejemplo, incluir metadatos sobre el documento del que se extrajo el texto).


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)


Necesitamos usar un modelo que admita la llamada a funciones/herramientas.

Usaremos .with_structured_output() para agregar las instrucciones de extracción a nuestro modelo de chat.

In [19]:
chain = prompt | llm.with_structured_output(schema=Person)
# Descripcion de la chain - el prompt se lo pasamos al LLM, pero este chat model
# va a utilizar un esquema definido por la clase persona

# Prueba

In [20]:
comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, USA"

chain.invoke({"text": comment})

Person(name='Sarah Johnson', lastname='Johnson', country='USA')

Notemos que solamente podemos encontrar datos de una sola entidad, en nuestro caso una sola persona

Tenga en cuenta que esta capacidad de extracción es generativa, lo que significa que nuestro modelo puede realizar diversas tareas más allá de lo esperado. Por ejemplo, el modelo podría inferir el género de un usuario a partir de su nombre, incluso si esta información no se proporciona explícitamente.

# Extracción de una lista de entidades en lugar de una sola entidad

En proyectos reales, probablemente trabajará con un texto extenso que incluya más de una reseña de usuario. Podemos extraer los datos clave de varios usuarios anidando modelos de Pydantic (**nesting Pydantic models**).

Observe cómo la definición del modelo de datos incluye el modelo Persona. Esto se denomina técnicamente "nesting" models. ("anidar modelos")

In [28]:
from typing import List

class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

chain = prompt | llm.with_structured_output(schema=Data) #Importante reescribir nuevamente la chain con la nueva estructura


Vea cómo ahora estamos usando el modelo de datos con el llm:

In [29]:
# Example input text that mentions multiple people
text_input = """
Alice Johnson from Canada recently reviewed a book she loved. Meanwhile, Bob Smith from the USA shared his insights on the same book in a different review. Both reviews were very insightful.
"""

# Invoke the processing chain on the text
response = chain.invoke({"text": text_input})

# Output the extracted data
response

Data(people=[Person(name='Alice Johnson', lastname=None, country='Canada'), Person(name='Bob Smith', lastname=None, country='USA')])